In [79]:
import pandas as pd
import numpy as np

In [80]:
data = pd.read_csv("korean_air_2020_01.csv")
#second = pd.read_csv("korean_air_data_korean_air_2020_02.csv")
#third = pd.read_csv("korean_air_data_korean_air_2020_07.csv")
#fourth = pd.read_csv("korean_air_data_korean_air_2020_08.csv")
#fifth = pd.read_csv("korean_air_data_korean_air_2020_09.csv")
#sixth = pd.read_csv("korean_air_data_korean_air_2020_10.csv")

#data = pd.concat([first, second, third, fourth, fifth, sixth])

In [81]:
data = data.dropna()
data.columns = ["a","title","text","repoter","spon", "date"]
date = data.date

In [82]:
#q불용어 지정
#기자이름
repoter = data["repoter"].unique

In [83]:
#신문사 이름
spon = data["spon"].unique

In [84]:
#대표적인 조사
stopwords = ['의','가','이','권','은','들','는','좀','잘','걍','과','도','을','것','를','등','으로','자','에','등','와','한','하다''것','로']

In [85]:
#불용어 병합
stopwords.extend(data.repoter.values)
stopwords.extend(data.spon.values)

In [86]:
len(data)

432

In [87]:
# 한글 외의 모든 데이터 제거 : 기업명이 영어로 되어있는 경우를 어떻게 처리 해야할까?
data = data.text.str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")

from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer

okt = Okt()
X_train = []
X_train2 =[]

for sen in data :
    temp_X = []
    
    # 토큰화
    temp_X = okt.morphs(sen, stem = True) 
    
    # 불용어 제거
    temp_X = [word for word in temp_X if not word in stopwords] 
    
    #모든 토큰들을 리스트에 기사별로 저장
    X_train.append(temp_X)
    
    #모든 토큰들을  하나의 리스트에 저장
    X_train2.extend(temp_X)

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

threshold = 3
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

# 전체 단어 개수 중 빈도수 2이하인 단어 개수는 제거.
vocab_size = total_cnt - rare_cnt
print('단어 집합의 크기 :',vocab_size)

단어 집합(vocabulary)의 크기 : 8592
등장 빈도가 2번 이하인 희귀 단어의 수: 4133
단어 집합에서 희귀 단어의 비율: 48.10288640595903
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 3.481598379020788
단어 집합의 크기 : 4459


In [88]:
import datetime
def getDayName(a):
    D = a.split("/")
    y = int(D[0])
    m = int(D[1])
    d = int(D[2])
    return ['MON','TUE','WED','THU','FRI','SAT','SUN'][datetime.date(y,m,d).weekday()]

In [89]:
weekday =[]
for x in date:
    weekday.append(getDayName(x))
    

In [90]:
date = date.tolist()

In [91]:
date

['2020/01/31',
 '2020/01/31',
 '2020/01/31',
 '2020/01/31',
 '2020/01/31',
 '2020/01/31',
 '2020/01/31',
 '2020/01/31',
 '2020/01/31',
 '2020/01/31',
 '2020/01/31',
 '2020/01/31',
 '2020/01/31',
 '2020/01/31',
 '2020/01/31',
 '2020/01/31',
 '2020/01/31',
 '2020/01/31',
 '2020/01/31',
 '2020/01/31',
 '2020/01/31',
 '2020/01/31',
 '2020/01/31',
 '2020/01/31',
 '2020/01/31',
 '2020/01/31',
 '2020/01/31',
 '2020/01/31',
 '2020/01/31',
 '2020/01/31',
 '2020/01/31',
 '2020/01/31',
 '2020/01/31',
 '2020/01/31',
 '2020/01/31',
 '2020/01/31',
 '2020/01/31',
 '2020/01/31',
 '2020/01/31',
 '2020/01/31',
 '2020/01/31',
 '2020/01/31',
 '2020/01/31',
 '2020/01/31',
 '2020/01/31',
 '2020/01/31',
 '2020/01/31',
 '2020/01/31',
 '2020/01/31',
 '2020/01/30',
 '2020/01/30',
 '2020/01/30',
 '2020/01/30',
 '2020/01/30',
 '2020/01/30',
 '2020/01/30',
 '2020/01/30',
 '2020/01/30',
 '2020/01/30',
 '2020/01/30',
 '2020/01/30',
 '2020/01/30',
 '2020/01/30',
 '2020/01/30',
 '2020/01/30',
 '2020/01/30',
 '2020/01/

In [92]:
DATE = []
for x in date:
    if x not in DATE:
        DATE.append(x)
print(DATE)

['2020/01/31', '2020/01/30', '2020/01/29', '2020/01/28', '2020/01/27', '2020/01/26', '2020/01/25', '2020/01/24', '2020/01/23', '2020/01/22', '2020/01/21', '2020/01/20', '2020/01/19', '2020/01/18', '2020/01/17', '2020/01/16', '2020/01/15', '2020/01/14', '2020/01/13', '2020/01/12', '2020/01/11', '2020/01/10', '2020/01/09', '2020/01/08', '2020/01/07', '2020/01/06', '2020/01/05', '2020/01/01']


In [125]:
#증감 데이터 불러오기
increase = pd.read_csv("대비처리.csv")

In [126]:
increase

,Unnamed: 0,년/월/일,종가,대비,거래량(주),거래대금(원),시가,고가,저가,시가총액(백만),상장주식수(주),증감
0,0,2020/09/29,"18,850",50,"922,389","17,470,132,150","19,150","19,150","18,800","3,283,853","174,209,713",0
1,1,2020/09/28,"18,800",450,"966,495","18,103,219,950","18,550","18,900","18,500","3,275,143","174,209,713",1
2,2,2020/09/25,"18,350",350,"895,129","16,449,493,950","18,100","18,600","18,100","3,196,748","174,209,713",1
3,3,2020/09/24,"18,000",-550,"1,234,760","22,462,251,500","18,400","18,450","18,000","3,135,775","174,209,713",-1
4,4,2020/09/23,"18,550",150,"1,071,051","19,859,025,200","18,600","18,850","18,400","3,231,590","174,209,713",0
...,...,...,...,...,...,...,...,...,...,...,...,...
182,182,2020/01/08,"26,300",-550,"676,198","17,787,052,350","26,450","26,700","25,950","2,494,414","94,844,634",-1
183,183,2020/01/07,"26,850",350,"388,103","10,409,787,950","26,500","27,150","26,400","2,546,578","94,844,634",1
184,184,2020/01/06,"26,500",-800,"690,224","18,261,902,900","26,850","27,100","26,000","2,513,383","94,844,634",-1
185,185,2020/01/03,"27,300",-550,"572,272","15,794,199,800","28,000","28,150","27,250","2,589,259","94,844,634",-1


In [94]:
increase = increase[["년/월/일","증감"]]

In [95]:
weekday_frame = pd.DataFrame( {"date" : date,"weekday" : weekday})

In [96]:
weekday_frame

,date,weekday
0,2020/01/31,FRI
1,2020/01/31,FRI
2,2020/01/31,FRI
3,2020/01/31,FRI
4,2020/01/31,FRI
...,...,...
427,2020/01/01,WED
428,2020/01/01,WED
429,2020/01/01,WED
430,2020/01/01,WED


In [97]:
 for x in range(0,len(weekday_frame)-1):
    if weekday_frame.weekday[x] == "SAT":
        monday = DATE.index(weekday_frame.date[x])- 2
        weekday_frame.date[x] = DATE[monday]
        weekday_frame.weekday[x] == "MON"
            
    else :
        if weekday_frame.weekday[x] == "SUN":
            monday = DATE.index(weekday_frame.date[x]) - 1
            weekday_frame.date[x] = DATE[monday]
            weekday_frame.weekday[x] == "MON"
                
        else : pass
                

In [98]:
weekday_frame.date.unique()

array(['2020/01/31', '2020/01/30', '2020/01/29', '2020/01/28',
       '2020/01/27', '2020/01/24', '2020/01/23', '2020/01/22',
       '2020/01/21', '2020/01/20', '2020/01/17', '2020/01/16',
       '2020/01/15', '2020/01/14', '2020/01/13', '2020/01/10',
       '2020/01/09', '2020/01/08', '2020/01/07', '2020/01/06',
       '2020/01/01'], dtype=object)

In [100]:
weekday_frame = weekday_frame.drop_duplicates()

In [101]:
weekday_frame

,date,weekday
0,2020/01/31,FRI
49,2020/01/30,THU
66,2020/01/29,WED
93,2020/01/28,TUE
133,2020/01/27,MON
143,2020/01/27,SUN
147,2020/01/27,SAT
149,2020/01/24,FRI
153,2020/01/23,THU
167,2020/01/22,WED


In [102]:
dic_frame

,data,date
0,"[남매, 난, 촉발, 되다, 한진, 그룹, 경영, 분쟁, 정점, 치닫다, 있다, 그...",2020/01/31
1,"[한진, 그룹, 경영, 향배, 가르다, 월, 한진, 칼, 주주총회, 앞두다, 조현아...",2020/01/31
2,"[줄어들다, 비행, 수당, 처우, 대한, 정보, 아직, 없다, 신종, 코로나바이러스...",2020/01/31
3,"[한진, 남매, 경영, 분쟁, 본격, 화, 입장, 문, 경영, 일선, 안, 나서다,...",2020/01/31
4,"[앵커, 오늘, 우한, 교민, 이송, 전세기, 에는, 조원태, 한진, 그룹, 회장,...",2020/01/31
...,...,...
427,"[성윤, 모, 오른쪽, 세번, 째, 산업, 통상, 자원부, 장관, 일, 인천공항, ...",2020/01/01
428,"[세종, 권혜민, 기자, 일본, 정부, 수출, 규제, 강화, 조치, 미국, 중국, ...",2020/01/01
429,"[경제, 기상도, 업종, 별, 전망, 항공, 국내, 항공, 업다, 올해, 침체, 기...",2020/01/01
430,"[삼, 성, 현, 기차, 국내, 주요, 대기업, 년, 내, 클라우드, 도입, 마무리...",2020/01/01


In [103]:
#날자를 중심으로 데이터 병합
increase = increase.rename(columns ={ "년/월/일" : "date"})

In [104]:
dic_frame = pd.DataFrame({"data" : X_train, "date" : date})

In [105]:
increase2 = pd.merge(weekday_frame, increase , on = "date")

In [107]:
del increase2["weekday"]

In [112]:
increase2 = increase2.drop_duplicates()

In [155]:
dic_frame2 = pd.merge( dic_frame, increase2, on = "date", how = "left")

In [172]:
dic_frame2 = dic_frame2.fillna(0)

In [136]:
#딕셔너리 key 생성
key_list = []
for v in X_train2:
    if v not in key_list:
        key_list.append(v)

In [154]:
len(dic_frame2)

385

In [175]:
dic_frame2.증감.unique()

array([ 0., -3., -2.,  1., -1.,  2.])

In [176]:
#증감을 기준으로 단어 정렬
dic = dic_frame2.drop("date",axis = 1)

In [177]:
dic

,data,증감
0,"[남매, 난, 촉발, 되다, 한진, 그룹, 경영, 분쟁, 정점, 치닫다, 있다, 그...",0.0
1,"[한진, 그룹, 경영, 향배, 가르다, 월, 한진, 칼, 주주총회, 앞두다, 조현아...",0.0
2,"[줄어들다, 비행, 수당, 처우, 대한, 정보, 아직, 없다, 신종, 코로나바이러스...",0.0
3,"[한진, 남매, 경영, 분쟁, 본격, 화, 입장, 문, 경영, 일선, 안, 나서다,...",0.0
4,"[앵커, 오늘, 우한, 교민, 이송, 전세기, 에는, 조원태, 한진, 그룹, 회장,...",0.0
...,...,...
427,"[성윤, 모, 오른쪽, 세번, 째, 산업, 통상, 자원부, 장관, 일, 인천공항, ...",0.0
428,"[세종, 권혜민, 기자, 일본, 정부, 수출, 규제, 강화, 조치, 미국, 중국, ...",0.0
429,"[경제, 기상도, 업종, 별, 전망, 항공, 국내, 항공, 업다, 올해, 침체, 기...",0.0
430,"[삼, 성, 현, 기차, 국내, 주요, 대기업, 년, 내, 클라우드, 도입, 마무리...",0.0


['국내',
 '에서',
 '신종',
 '코로나바이러스',
 '감염증',
 '인',
 '우한',
 '폐렴',
 '네',
 '번째',
 '확진',
 '환자',
 '발생',
 '가운데',
 '일',
 '오후',
 '서울',
 '중구',
 '명동',
 '하다',
 '약국',
 '에서',
 '외국인',
 '관광객',
 '마스크',
 '사기',
 '위해',
 '줄',
 '서고',
 '있다',
 '연합뉴스',
 '우한',
 '폐렴',
 '불리다',
 '신종',
 '코로나',
 '바이러스',
 '감염증',
 '국내',
 '확진',
 '환자',
 '추가',
 '발생',
 '하다',
 '유통업',
 '관광업',
 '계',
 '에도',
 '공포',
 '감',
 '확산',
 '되다',
 '있다',
 '온라인',
 '쇼핑몰',
 '검색어',
 '상위',
 '중국',
 '여행',
 '취소',
 '관광객',
 '속출',
 '항공사',
 '환불',
 '수수료',
 '면제',
 '온라인',
 '쇼핑몰',
 '에서',
 '마스크',
 '손',
 '세',
 '정제',
 '구입',
 '하다',
 '소비자',
 '늘',
 '고',
 '있다',
 '지마켓',
 '캡처',
 '설',
 '연휴',
 '지마켓',
 '옥션',
 '온라인',
 '쇼핑몰',
 '에는',
 '대량',
 '마스크',
 '제품',
 '구매',
 '하다',
 '소비자',
 '몰리다',
 '지마켓',
 '베스트',
 '상품',
 '카테고리',
 '개',
 '중',
 '개',
 '제외',
 '개일',
 '오후',
 '시',
 '기준',
 '늘다',
 '마스크',
 '손',
 '세',
 '정제',
 '제품',
 '차지',
 '하다',
 '옥션',
 '베스트',
 '상품',
 '개',
 '중',
 '개',
 '마스크',
 '제품',
 '이다',
 '온라인',
 '쇼핑몰',
 '인기',
 '검색어',
 '로는',
 '마스크',
 '관련',
 '단어',
 '주',
 '이루다',
 '또',
 '마스크',
 '외',
 '에도',
 '손',
 '

In [179]:
#예상 소요 시간 300분
#사전 만들기
dictionary = {"<UNK>" : { "횟수" : 0, "점수" : 0, "극성정도" : 0}}

for k in key_list:  #키 리스트안에 있는 단어가 언급이 되는 경우
    count = 0
    score = 0
    for x in range(0,len(dic)):  #한 기사씩 검사한다.
        d = dic.loc[x,:].data
        
        
        #count 는 전체 언급된 횟수, score는 증감의 총합, pority는 총합의 평균
        if k in d:            
            count = count + d.count(k)
            score = score + (dic.loc[x,:]["증감"] * d.count(k))
        
       
        #elif k not in word_list:
        else:
            pass

    polarity  = score/count
    dictionary[k] = { "횟수" : count, "점수" : score, "극성정도" : polarity}  

In [180]:
len(dictionary)

8593

In [181]:
dictionary

{'<UNK>': {'횟수': 0, '점수': 0, '극성정도': 0},
 '남매': {'횟수': 91, '점수': 12.0, '극성정도': 0.13186813186813187},
 '난': {'횟수': 31, '점수': -8.0, '극성정도': -0.25806451612903225},
 '촉발': {'횟수': 7, '점수': 0.0, '극성정도': 0.0},
 '되다': {'횟수': 1605, '점수': -471.0, '극성정도': -0.29345794392523367},
 '한진': {'횟수': 1602, '점수': -197.0, '극성정도': -0.1229712858926342},
 '그룹': {'횟수': 912, '점수': -138.0, '극성정도': -0.1513157894736842},
 '경영': {'횟수': 899, '점수': -115.0, '극성정도': -0.12791991101223582},
 '분쟁': {'횟수': 219, '점수': -37.0, '극성정도': -0.1689497716894977},
 '정점': {'횟수': 2, '점수': 0.0, '극성정도': 0.0},
 '치닫다': {'횟수': 7, '점수': -1.0, '극성정도': -0.14285714285714285},
 '있다': {'횟수': 2317, '점수': -780.0, '극성정도': -0.3366422097539922},
 '향배': {'횟수': 13, '점수': -2.0, '극성정도': -0.15384615384615385},
 '가늠': {'횟수': 8, '점수': 3.0, '극성정도': 0.375},
 '하다': {'횟수': 6491, '점수': -2036.0, '극성정도': -0.3136650747188415},
 '월': {'횟수': 728, '점수': -223.0, '극성정도': -0.30631868131868134},
 '칼': {'횟수': 816, '점수': -120.0, '극성정도': -0.14705882352941177},
 '주주총회': {'횟수': 